### Setup

In [1]:
import plotly.graph_objects as go
import plotly.io as pio

# Define the updated base template
base = go.layout.Template(
    layout=go.Layout(
        paper_bgcolor='#FFF5CC',
        plot_bgcolor='#FFF5CC',
        height=800,
        width=800 * 1.618,
        xaxis=dict(
            anchor='y',
            showgrid=True,
            gridcolor='#888888',  # Darker grid lines
            tickfont=dict(
                size=24,
                family='Open Sans, sans-serif'  # Use Open Sans font
            ),
            titlefont=dict(
                size=26,
                family='Open Sans, sans-serif'  # Use Open Sans font
            ),
            linecolor='#333333',
            linewidth=2  # Adjust the thickness of the x-axis line
        ),
        yaxis=dict(
            anchor='x',
            showgrid=True,
            gridcolor='#888888',  # Darker grid lines
            tickfont=dict(
                size=24,
                family='Open Sans, sans-serif'
            ),
            titlefont=dict(
                size=26,
                family='Open Sans, sans-serif'
            ),
            linecolor='#333333',
            linewidth=2  # Adjust the thickness of the y-axis line
        ),
        font=dict(
            color='#333333',
            size=28,
            family='Open Sans, sans-serif'
        ),
        # Updated colorway to ensure more distinguishable colors
        colorway=["#470945", # H: Violet
                  "#E67E5A", # H: Orange (Sienna)
                  "#297FB9", # H: Blue (Steel)
                  "#163748", # D: Charcoal
                  "#4F1787", # H: Purple
                  "#EFE04E", # H: Yellow (Maize)
                  "#214F70", # D: Indigo
                  "#DF14AA", # H: Pink (Cerise)
                  "#100B1A", # D: Black
                  "#12C4CF", # H: Teal
                  "#14193D", # D: Space
                  "#CC5500"],# H: Cream
        title=go.layout.Title(
            text='',
            font=dict(
                size=34,
                color='#333333',
                family='Open Sans, sans-serif'
            ),
            x=0.05,
        )
    ),
    data=dict(
        scatter=[
            go.Scatter(
                line=dict(width=3)  # Set the line width for scatter plots
            )
        ]
    )
)

# Register the updated base template
pio.templates['base'] = base
pio.templates.default = 'base'

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def fetch_population_data(url="https://www.worldometers.info/world-population/population-by-country/"):
    """
    Fetch population data and additional statistics from Worldometer and return it as a pandas DataFrame.

    Args:
        url (str): URL of the Worldometer population page.

    Returns:
        pd.DataFrame: DataFrame with columns ['country', 'population', 'yearly_change', 'net_change',
                                               'density', 'land_area', 'migrants', 'fert_rate', 
                                               'med_age', 'urban_pop_perc', 'world_share'].
    """
    # Send a request to the website
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for HTTP errors

    # Parse the webpage content
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the table containing the population data
    table = soup.find('table', {'id': 'example2'})

    # Extract table rows
    rows = table.find_all('tr')

    # Extract header and data
    data = []
    for row in rows[1:]:  # Skip the header row
        cols = row.find_all('td')
        country = cols[1].text.strip()
        population = int(cols[2].text.strip().replace(',', ''))
        yearly_change_perc = cols[3].text.strip().replace(' %', '')  # Remove space and % symbol
        net_change = int(cols[4].text.strip().replace(',', ''))  # Net Change
        density = int(cols[5].text.strip().replace(',', ''))  # Density (P/Km²)
        land_area = int(cols[6].text.strip().replace(',', ''))  # Land Area (Km²)
        migrants = cols[7].text.strip()  # Migrants (net)
        fert_rate = cols[8].text.strip()  # Fertility Rate
        med_age = cols[9].text.strip()  # Median Age
        urban_pop_perc = cols[10].text.strip().replace(' %', '')  # Remove space and % symbol
        world_share_perc = cols[11].text.strip().replace(' %', '')  # Remove space and % symbol
        data.append([country, population, yearly_change_perc, net_change, density, land_area, 
                     migrants, fert_rate, med_age, urban_pop_perc, world_share_perc])

    # Create a DataFrame
    df = pd.DataFrame(data, columns=['country', 'population', 'yearly_change_perc', 'net_change', 'density',
                                     'land_area', 'migrants', 'fert_rate', 'med_age', 
                                     'urban_pop_perc', 'world_share_perc'])

    # Convert cleaned percentage columns to float, handling 'N.A.' gracefully
    df['yearly_change_perc'] = pd.to_numeric(df['yearly_change_perc'], errors='coerce')
    df['urban_pop_perc'] = pd.to_numeric(df['urban_pop_perc'], errors='coerce')
    df['world_share_perc'] = pd.to_numeric(df['world_share_perc'], errors='coerce')

    return df

# Fetch the population data with additional columns
population_data = fetch_population_data()

# Display the first few rows
print(population_data.head())

# Save to a CSV if needed
population_data.to_csv("d_country_population_raw.csv", index=False)


/Users/arya/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


         country  population  yearly_change_perc  net_change  density  \
0          India  1450935791                0.89    12866195      488   
1          China  1419321278               -0.23    -3263655      151   
2  United States   345426571                0.57     1949236       38   
3      Indonesia   283487931                0.82     2297864      156   
4       Pakistan   251269164                1.52     3764669      326   

   land_area    migrants fert_rate med_age  urban_pop_perc  world_share_perc  
0    2973190    -630,830       2.0      28            37.0             17.78  
1    9388211    -318,992       1.0      40            66.0             17.39  
2    9147420   1,286,132       1.6      38            82.0              4.23  
3    1811570     -38,469       2.1      30            59.0              3.47  
4     770880  -1,401,173       3.5      20            34.0              3.08  


In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import unicodedata

def fetch_iso_country_codes(url="https://en.wikipedia.org/wiki/List_of_ISO_3166_country_codes"):
    """
    Fetch ISO country codes from Wikipedia and return them as a pandas DataFrame, removing any notes in square brackets.

    Args:
        url (str): URL of the Wikipedia page for ISO 3166 country codes.

    Returns:
        pd.DataFrame: DataFrame with columns ['country_name', 'official_name', 'iso_alpha2', 'iso_alpha3', 'numeric_code', 'tld'].
    """
    # Send a request to the website
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for HTTP errors

    # Parse the webpage content
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all tables on the page
    tables = soup.find_all('table')

    # Locate the correct table by inspecting its position or content
    target_table = None
    for table in tables:
        if "ISO 3166" in table.text:  # Check if the table contains relevant text
            target_table = table
            break

    if target_table is None:
        raise ValueError("Unable to find the ISO 3166 country codes table.")

    # Extract table rows
    rows = target_table.find_all('tr')

    # Extract header and data
    data = []
    for row in rows[1:]:  # Skip the header row
        cols = row.find_all('td')
        if len(cols) >= 6:  # Ensure the row has enough columns
            country_name = cols[0].text.strip().replace('\n', '')
            official_name = cols[1].text.strip().replace('\n', '')
            iso_alpha2 = cols[3].text.strip()
            iso_alpha3 = cols[4].text.strip()
            numeric_code = cols[5].text.strip()
            tld = cols[6].text.strip()

            # Remove notes (e.g., [f], [b], [c][d]) and clean special characters
            country_name = pd.Series([country_name]).str.replace(r'\[.*?\]', '', regex=True).str.strip().values[0]
            official_name = pd.Series([official_name]).str.replace(r'\[.*?\]', '', regex=True).str.strip().values[0]

            # Normalize text to remove any hidden special characters
            country_name = unicodedata.normalize('NFKC', country_name)
            official_name = unicodedata.normalize('NFKC', official_name)

            data.append([country_name, official_name, iso_alpha2, iso_alpha3, numeric_code, tld])

    # Create a DataFrame
    df = pd.DataFrame(data, columns=['country_name', 'official_name', 'iso_alpha2', 'iso_alpha3', 'numeric_code', 'tld'])

    return df

iso_country_codes = fetch_iso_country_codes()

# Save to a CSV if needed
iso_country_codes.to_csv("d_country_codes.csv", index=False)


### Miss Universe

In [44]:
import pandas as pd
import plotly.graph_objects as go

def plot_top_five_finishes(min_top_five=1):
    """
    Plot the number of Top Five Finishes by Country in Miss Universe with country flags.

    Args:
        min_top_five (int): Minimum number of top-five finishes to include in the chart.
        
    Returns:
        pd.DataFrame: A dataframe containing the filtered results.
    """
    # Load the dataset, handling varying column counts
    df = pd.read_csv("d_miss_universe.csv", header=0, names=['year', 'first', 'second', 'third', 'fourth', 'fifth'], engine='python')

    # Combine all the top-five columns into one list, excluding "-"
    top_five_countries = pd.concat([
        df['first'], df['second'], df['third'], df['fourth'], df['fifth']
    ]).dropna()
    top_five_countries = top_five_countries[top_five_countries != "-"]

    # Count the occurrences of each country
    top_five_counts = top_five_countries.value_counts().reset_index()
    top_five_counts.columns = ['country', 'count']

    # Filter by minimum number of top-five finishes
    filtered_counts = top_five_counts[top_five_counts['count'] >= min_top_five]

    # Replace country codes with flag emojis
    country_flags = {
        "USA": "🇺🇸", "VEN": "🇻🇪", "BRA": "🇧🇷", "COL": "🇨🇴", "GBR": "🇬🇧",
        "SWE": "🇸🇪", "PHL": "🇵🇭", "FIN": "🇫🇮", "PRI": "🇵🇷", "AUS": "🇦🇺",
        "MEX": "🇲🇽"  # Add other codes and flags as necessary
    }
    filtered_counts['country'] = filtered_counts['country'].map(lambda x: country_flags.get(x, x))

    # Create the bar chart using Plotly
    fig = go.Figure()

    fig.add_trace(go.Bar(
        x=filtered_counts['country'],
        y=filtered_counts['count'],
        text=filtered_counts['count'],  # Show counts on bars
        textposition='outside',  # Place text outside the bars
        textfont=dict(size=40)  # Make the numbers on top of bars larger
    ))

    # Update layout
    fig.update_layout(
        template='base',  # Ensure the 'base' template is registered
        title=None,
        xaxis=dict(
            showticklabels=True,
            tickfont=dict(size=50),  # Make axis values bigger
        ),
        yaxis=dict(
            showticklabels=False,
            tickfont=dict(size=30),  # Make axis values bigger
            range=[0, filtered_counts['count'].max() + 5]  # Add padding above the bars
        ),
        showlegend=False,
        xaxis_showgrid=False,  # Remove vertical gridlines
        yaxis_showgrid=False,   # Remove horizontal gridlines
        width=1280,  # Set width for 16:9 ratio
        height=720   # Set height for 16:9 ratio
    )

    # Hide axis titles
    fig.update_xaxes(title=None)
    fig.update_yaxes(title=None)

    # Show the plot
    fig.show()

    return filtered_counts

# Example usage: Adjust the minimum number of top finishes to include
result = plot_top_five_finishes(min_top_five=10)


/var/folders/r0/4vr5j1517ggd5nvq86w27hhm0000gn/T/ipykernel_2090/3648987695.py:36: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [49]:
import pandas as pd
import plotly.graph_objects as go

def top_five_per_million(min_top_five=1):
    """
    Calculate and plot Top Five Finishes Per 1 Million Inhabitants by Country with consistent formatting and a custom order.
    
    Args:
        min_top_five (int): Minimum number of top-five finishes to include in the chart.
        
    Returns:
        pd.DataFrame: A dataframe containing the filtered results.
    """
    # Load datasets
    miss_universe_df = pd.read_csv("d_miss_universe.csv", header=0, names=['year', 'first', 'second', 'third', 'fourth', 'fifth'], engine='python')
    population_df = pd.read_csv("d_country_population.csv")  # Ensure it contains columns 'country_code' and 'population'
    
    # Combine all top-five columns into one list, excluding "-"
    top_five_countries = pd.concat([
        miss_universe_df['first'], miss_universe_df['second'], miss_universe_df['third'], miss_universe_df['fourth'], miss_universe_df['fifth']
    ]).dropna()
    top_five_countries = top_five_countries[top_five_countries != "-"]
    
    # Count the number of top-five finishes by country
    top_five_counts = top_five_countries.value_counts().reset_index()
    top_five_counts.columns = ['country_code', 'top_five_count']
    
    # Merge the top-five counts with the population data
    merged_df = pd.merge(top_five_counts, population_df, on='country_code', how='inner')
    
    # Calculate Top Five Finishes Per 1 Million Inhabitants
    merged_df['top_five_per_million'] = (merged_df['top_five_count'] / merged_df['population']) * 1_000_000
    
    # Filter by minimum number of top-five finishes
    filtered_df = merged_df[merged_df['top_five_count'] >= min_top_five]
    
    # Replace country codes with flag emojis
    country_flags = {
        "USA": "🇺🇸", "VEN": "🇻🇪", "BRA": "🇧🇷", "COL": "🇨🇴", "GBR": "🇬🇧",
        "SWE": "🇸🇪", "PHL": "🇵🇭", "FIN": "🇫🇮", "PRI": "🇵🇷", "AUS": "🇦🇺",
        "MEX": "🇲🇽"  # Add other codes and flags as necessary
    }
    filtered_df['country'] = filtered_df['country_code'].map(lambda x: country_flags.get(x, x))

    # Enforce custom sort order
    custom_order = ["USA", "VEN", "BRA", "COL", "GBR", "SWE", "PHL", "FIN", "PRI", "AUS", "MEX"]
    filtered_df['country_code'] = pd.Categorical(filtered_df['country_code'], categories=custom_order, ordered=True)
    filtered_df = filtered_df.sort_values('country_code')

    # Plot the results
    fig = go.Figure()

    fig.add_trace(go.Bar(
        x=filtered_df['country'],
        y=filtered_df['top_five_per_million'],
        text=filtered_df['top_five_per_million'].round(2),  # Show per-million values on bars
        textposition='outside',  # Place text outside the bars
        textfont=dict(size=45)  # Make the numbers on top of bars larger
    ))

    # Update layout
    fig.update_layout(
        template='base',  # Ensure the 'base' template is registered
        title=None,
        xaxis=dict(
            showticklabels=True,
            tickfont=dict(size=50),  # Make axis values bigger
        ),
        yaxis=dict(
            showticklabels=False,
            tickfont=dict(size=30),  # Make axis values bigger
            range=[0, filtered_df['top_five_per_million'].max() + 0.5]  # Add padding above the tallest bar
        ),
        showlegend=False,
        xaxis_showgrid=False,  # Remove vertical gridlines
        yaxis_showgrid=False,  # Remove horizontal gridlines
        width=1280,  # Set width for 16:9 ratio
        height=720   # Set height for 16:9 ratio
    )

    # Hide axis titles
    fig.update_xaxes(title=None)
    fig.update_yaxes(title=None)

    # Show the plot
    fig.show()
    
    return filtered_df

# Example usage: Adjust the minimum number of top finishes to include
result = top_five_per_million(min_top_five=10)


/var/folders/r0/4vr5j1517ggd5nvq86w27hhm0000gn/T/ipykernel_2090/1447905813.py:43: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/r0/4vr5j1517ggd5nvq86w27hhm0000gn/T/ipykernel_2090/1447905813.py:47: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### Models

In [82]:
import pandas as pd
import plotly.graph_objects as go

def plot_models_social_data(x_axis, y_axis, top_n=None, exclude=None):
    """
    Plots a bar chart for the models' social data based on the selected x_axis and y_axis, with country flags.

    Parameters:
    - x_axis (str): The column to group by ('nationality' or 'ethnicity').
    - y_axis (str): The metric to plot ('count' for number of models or 'sum' for total followers).
    - top_n (int, optional): Number of top results to include. If None, includes all results.
    - exclude (str, optional): A value to exclude from the results (e.g., "USA").
    """
    # Load the dataset
    df = pd.read_csv("d_models_social.csv")
    
    # Filter out the excluded value if specified
    if exclude:
        df = df[df[x_axis] != exclude]
    
    # Group by the selected x_axis
    if y_axis == 'count':
        grouped_data = df.groupby(x_axis).size().reset_index(name='count')
    elif y_axis == 'sum':
        grouped_data = df.groupby(x_axis)['num_followers (MM)'].sum().reset_index(name='sum')
    else:
        raise ValueError("y_axis must be either 'count' or 'sum'")
    
    # Set appropriate y-axis label
    y_col = 'count' if y_axis == 'count' else 'sum'
    y_label = 'Number of Models' if y_axis == 'count' else 'Total Followers (MM)'
    
    # Sort the data in descending order
    grouped_data = grouped_data.sort_values(by=y_col, ascending=False)
    
    # Include only the top N results if specified
    if top_n:
        grouped_data = grouped_data.head(top_n)
    
    # Replace country names with flags for specific countries
    country_flags = {
        "American": "🇺🇸", 
        "British": "🇬🇧", 
        "Brazilian": "🇧🇷", 
        "Spanish": "🇪🇸", 
        "German": "🇩🇪",
    }
    grouped_data[x_axis] = grouped_data[x_axis].map(lambda x: country_flags.get(x, x))
    
    # Create the bar chart
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=grouped_data[x_axis],
        y=grouped_data[y_col],
        text=grouped_data[y_col].round(2) if y_axis == 'sum' else grouped_data[y_col],  # Add text on bars
        textposition='outside',  # Position text outside bars
        textfont=dict(size=45)  # Make text on bars larger
    ))
    
    # Update layout with consistent formatting
    fig.update_layout(
        template='base',
        title=None,  # No title
        xaxis=dict(
            showticklabels=True,
            tickfont=dict(size=50),  # Make axis values larger
        ),
        yaxis=dict(
            showticklabels=False,  # Hide y-axis tick labels
            tickfont=dict(size=30),  # Ensure consistent font size
            range=[0, grouped_data[y_col].max() * 1.1]  # Add padding above tallest bar to fit text
        ),
        showlegend=False,
        xaxis_showgrid=False,  # Remove vertical gridlines
        yaxis_showgrid=False,  # Remove horizontal gridlines
        width=1280,  # Set width for 16:9 aspect ratio
        height=720   # Set height for 16:9 aspect ratio
    )

    # Hide axis titles
    fig.update_xaxes(title=None)
    fig.update_yaxes(title=None)

    # Show the plot
    fig.show()

# Example usage
# Plotting top 5 ethnicities based on total followers, excluding "USA
plot_models_social_data(x_axis='nationality', y_axis='sum', top_n=4, exclude='American')
plot_models_social_data(x_axis='nationality', y_axis='sum', top_n=5)
plot_models_social_data(x_axis='ethnicity', y_axis='sum', top_n=5)


### Porn

In [9]:
import pandas as pd
import plotly.graph_objects as go

def plot_porn_categories_chart(file_path, included_categories=None):
    """
    Plots a bar chart of porn categories and their number of videos.

    Parameters:
    - file_path (str): Path to the CSV file containing the data.
    - included_categories (list): List of categories to include. Default is None (include all).
    """
    # Load the dataset
    df = pd.read_csv(file_path)

    # Filter by included categories if provided
    if included_categories:
        df = df[df['category'].isin(included_categories)]

    # Sort the DataFrame by 'num_videos' in descending order
    df = df.sort_values(by='num_videos', ascending=False)

    # Create the bar chart using Plotly
    fig = go.Figure()

    fig.add_trace(go.Bar(
        x=df['category'],
        y=df['num_videos']
    ))

    # Update layout with the 'base' template and axis labels
    fig.update_layout(
        template='base',
        title='Number of Videos by Porn Category',
        xaxis_title='Category',
        yaxis_title='Number of Videos',
        xaxis=dict(type='category', tickangle=-45),
    )

    # Show the plot
    fig.show()

# Example usage:
# Plot all categories
plot_porn_categories_chart('d_porn_categories.csv')

# Plot specific categories
plot_porn_categories_chart('d_porn_categories.csv', included_categories=['Arab', 
                                                                         'Asian', 
                                                                         'BBW', 
                                                                         'Babysitter', 
                                                                         'Big Tits', 
                                                                         'Big Ass', 
                                                                         'Blonde',
                                                                         'Brazilian',
                                                                         'British',
                                                                         'Brunette',
                                                                         'Celebrity'
                                                                         'College (18+)',
                                                                         'Czech',
                                                                         'Ebony',
                                                                         'Euro',
                                                                         'Feet',
                                                                         'French',
                                                                         'German',
                                                                         'Indian',
                                                                         'Italian',
                                                                         'Japanese',
                                                                         'Korean',
                                                                         'Latina',
                                                                         'MILF',
                                                                         'Red Head',
                                                                         'Russian',
                                                                         'School (18+)'
                                                                         'Small Tits',
                                                                         'Step Fantasy',
                                                                         'Tattooed Women',
                                                                         'Teen (18+)'
                                                                        ])

plot_porn_categories_chart('d_porn_categories.csv', included_categories=['Arab', 
                                                                         'Asian', 
                                                                         'Blonde',
                                                                         'Brazilian',
                                                                         'British',
                                                                         'Brunette',
                                                                         'Czech',
                                                                         'Ebony',
                                                                         'Euro',
                                                                         'French',
                                                                         'German',
                                                                         'Indian',
                                                                         'Italian',
                                                                         'Japanese',
                                                                         'Korean',
                                                                         'Latina',
                                                                         'Red Head',
                                                                         'Russian',
                                                                         'Tattooed Women',
                                                                        ])


In [10]:
import pandas as pd

def calculate_total_watch_time(csv_file, view_percentage):
    """
    Calculate the total watch time and total views from a CSV file.
    
    Parameters:
    - csv_file: str, path to the CSV file.
    - view_percentage: float, the average percentage of the video watched (e.g., 0.75 for 75%).
    
    Returns:
    - dict: Total watch time in hours, days, weeks, months, years, and total views.
    """
    # Load the CSV
    df = pd.read_csv(csv_file)
    
    # Ensure no trailing/leading spaces in column names
    df.columns = df.columns.str.strip()
    
    # Check if the 'duration' column exists
    if 'duration' not in df.columns:
        raise KeyError("The 'duration' column is missing in the CSV file.")
    
    # Convert duration to seconds
    def duration_to_seconds(duration):
        parts = duration.split(':')
        seconds = int(parts[0]) * 60 + int(parts[1])  # Convert minutes and seconds
        return seconds

    df['duration_seconds'] = df['duration'].apply(duration_to_seconds)
    
    # Convert views to total views
    if 'views_mm' not in df.columns:
        raise KeyError("The 'views_mm' column is missing in the CSV file.")
    
    df['views'] = df['views_mm'] * 1_000_000
    
    # Calculate total watch time in seconds
    df['watch_time_seconds'] = df['views'] * df['duration_seconds'] * view_percentage
    
    # Sum the watch time across all videos
    total_watch_time_seconds = df['watch_time_seconds'].sum()
    
    # Convert total watch time to various units
    total_watch_time_minutues = total_watch_time_seconds / 60
    total_watch_time_hours = total_watch_time_seconds / 3600
    total_watch_time_days = total_watch_time_hours / 24
    total_watch_time_weeks = total_watch_time_days / 7
    total_watch_time_months = total_watch_time_days / 30.44  # Average days in a month
    total_watch_time_years = total_watch_time_days / 365.25  # Including leap years
    
    # Calculate total views
    total_views = df['views'].sum()
    
    return {
        "minutes": total_watch_time_minutues,
        "hours": total_watch_time_hours,
        "days": total_watch_time_days,
        "weeks": total_watch_time_weeks,
        "months": total_watch_time_months,
        "years": total_watch_time_years,
        "total_views": total_views
    }

# Example usage
csv_file = "d_porn_top50.csv"  # Path to your CSV file
view_percentage = 0.5  # 75% of each video is watched on average

try:
    total_watch_time = calculate_total_watch_time(csv_file, view_percentage)
    print(f"Total Watch Time: {total_watch_time['minutes']:.2f} minutes")
    print(f"Total Watch Time: {total_watch_time['hours']:.2f} hours")
    print(f"Total Watch Time: {total_watch_time['days']:.2f} days")
    print(f"Total Watch Time: {total_watch_time['weeks']:.2f} weeks")
    print(f"Total Watch Time: {total_watch_time['months']:.2f} months")
    print(f"Total Watch Time: {total_watch_time['years']:.2f} years")
    print(f"Total Views: {total_watch_time['total_views']:,} views")
except KeyError as e:
    print(e)

Total Watch Time: 42133940000.00 minutes
Total Watch Time: 702232333.33 hours
Total Watch Time: 29259680.56 days
Total Watch Time: 4179954.37 weeks
Total Watch Time: 961224.72 months
Total Watch Time: 80108.64 years
Total Views: 4,902,200,000.0 views
